# EA979 Final Project:
## Pedestrian Detection via ____

### Authors:
<ul><li>André Barros de Medeiros</li></ul>
<ul><li>Francesco Carella</li></ul>
<ul><li>Luca Genovese</li></ul>

The cell below executes all neccessary processing on the frames:

In [4]:
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt





# Video Capture 
#capture = cv2.VideoCapture(0) # use for camera stream
capture = cv2.VideoCapture("/Users/Andre/Documents/GitHub/pedestrian_recognition/method4/pedestrian.mp4") #use for video 
                                                            #file (insert correct path, path format changes based on OS)


# Keeps track of what frame we're on
frameCount = 0
Frame=[]
new_frames=[]

while(1):
    
    # Return Value and the current frame
    ret, frame = capture.read()

    #  Check if a current frame actually exist
    if not ret:
        break

    frameCount += 1
    # Resize the frame
    grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resizedFrame = cv2.resize(grayFrame, (0, 0), fx=1.5, fy=1.50)
    
    # Create an array in which I keep track of previous frames
    Frame.append(resizedFrame)
   
    #print(frameCount)
    
capture.release()
cv2.destroyAllWindows()

images = np.stack(Frame, axis=2)

dim=np.shape(images[1])
high=dim[0]
lenght=dim[1]

median=np.median(images,axis=2,keepdims=(high ,lenght))
background_median=median[:,:,0]  #It's an estimate of the background through median estimate

avg=np.average(images, axis=2)    #It's an estimate of the backgrount through average

l=len(Frame)
zeros=np.zeros((high,lenght), dtype=int )
Frame_median=[zeros]*l
Frame_avg=[zeros]*l

for i in range(l):
    
    img = Frame[i]
    
    image_median_filter = (img - background_median)**2//255
    new_frame_median_filter = np.where(image_median_filter > 10, 1, 0) #I have filtered the frames subtracting the background
    
    #image_avg_filter = (img - avg)**2//255
    #new_frame_avg_filter = np.where(image_avg_filter > 2, img, 0)
    
    Frame_median[i] = new_frame_median_filter
    #Frame_avg[i] = image_avg_filter
    
    #cv2.imshow('Frame', image_median_filter)
    
    
    k = cv2.waitKey(1) & 0xff
    if k == 27:
        break

cv2.destroyAllWindows()

l=len(Frame_median)

zeros=np.zeros((high,lenght), dtype=int )
Frame_with_square=[zeros]*l

for i in range(l):
    
    image=Frame_median[i]
    img_int=image.astype(np.uint8)*255
    
    Expanded_shape=cv2.dilate(img_int, None, iterations=3) 
    shape, _ = cv2.findContours( Expanded_shape, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
    
    for j in shape:
            (x1, y1, x2, y2) = cv2.boundingRect(j)
        
            if cv2.contourArea(j) < 900:         # Here I can decide what's the max area of a white shape
                continue
            
            cv2.rectangle(Expanded_shape, (x1-5, y1-5), (x1+x2+5, y1+y2+5), 255, -1)
            
    Frame_with_square[i]=Expanded_shape        
            

    

zeros=np.zeros((high,lenght), dtype=int )
Final_modified_video=[zeros]*l

for i in range(l):
    
    img = Frame_with_square[i]
    image = Frame[i]
    #img = Frame_avg[i]
    new_frame = np.where(img > 2, image, 0)
    Final_modified_video[i] = new_frame

    

cv2.destroyAllWindows()



ValueError: too many values to unpack (expected 2)

The cell below reproduces the processed frames from the above cell in video format:

In [2]:
for i in range(l):
    
    img = Final_modified_video[i]
    #img = Frame_avg[i]
    cv2.imshow('Frame', img)
    
    time.sleep(0.02)

    k = cv2.waitKey(1) & 0xff
    if k == 27:
        break

cv2.destroyAllWindows()